# ToDo List!
* ¿Qué suposición hace el MAE? ¿Qué está minimizando? ¿Por qué conviene usarlo como función de costo en este caso?
* Usar otros optimizadores

# Fuentes

### Link: https://heartbeat.fritz.ai/5-regression-loss-functions-all-machine-learners-should-know-4fb140e9d4b0
En esta fuente se puede encontrar una breve explicación del MAE y del MSE, una comparación entre ambos respecto de su comportamiento en entrenamiento frente a conjuntos de datos con y sin outliers, y luego una comparación de su comportamiento durante entrenamiento a razón de cómo son sus gradientes, lo cual provoca en el caso del MAE que la convergencia sea más lenta y sea necesario utilizar un **learning rate dinámico**. Explica que, si nos importa que la presencia de outliers tenga un impacto directo sobre el modelo, deberíamos utilizar MSE, mientras que si deseamos que no afecte demasiado podemos emplear MAE.

### Link: https://towardsdatascience.com/learning-rate-schedules-and-adaptive-learning-rate-methods-for-deep-learning-2c8f433990d1
En esta fuente se puede encontrar una explicación de los tres métodos para learning rate dinámico utilizados, el **time-based decay**, el **step decay** y el **exponential decay**, empleando para algunos de ellos la clase de Keras llamada Learning Rate Scheduler, que permite modificar a gusto del usuario el valor del learning rate a través del proceso.

### Link: https://stackoverflow.com/questions/46308374/what-is-validation-data-used-for-in-a-keras-sequential-model
Esta disución de StackOverflow es interesante sobre la separación de los datasets en entrenamiento, validación y evaluación del modelo, la use para verificar algunas cuestiones sobre cómo usaba la información de validación Keras, entre otras cosas.

### Link: https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/
Explicación sobre el uso de **early stopping**, donde básicamente buscamos parar el entrenamiento aunque no se hayan terminado de correr todos los epochs predefinidos, porque se detecta que no hay mejoría en los resultados obtenidos, para ello se emplea la métrica evaluada sobre el conjunto de validación.

### Link: https://machinelearningmastery.com/polynomial-features-transforms-for-machine-learning/
Explicación sobre el uso de **features polinomiales**, que básicamente consiste en agregar nuevas variables de entrada al modelo a partir de potencias obtenidas entre las variables de entrada originales. De esta forma, el espacio que conforman las variables es de mayor dimensión y por ello la solución es más flexible, aunque hay que tener cuidado de que no se ajuste demasiado provocando **overfitting**.

# 1. Cargando base de datos

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
import numpy as np

In [3]:
import importlib

In [4]:
import sys

In [5]:
sys.path.insert(0, '../..')

In [6]:
# Read the database from the .csv file into a pandas dataframe
df = pd.read_csv('../../databases/insurance.csv')

# 2. Preprocesamiento de los datos

In [7]:
from sklearn import preprocessing

## 2.1. Codificación de variables no numéricas

In [8]:
# Create a label encoder for the sex variable or feature and create a new column in the dataframe 
# with the encoded version of the gender
sex_encoder = preprocessing.LabelEncoder()
sex_encoder.fit(df['sex'])
df['sex-encoded'] = sex_encoder.transform(df['sex'])

In [9]:
# Create a label encoder for the smoker variable or feature and create a new column in the dataframe
# with the encoded version of the smoker
smoker_encoder = preprocessing.LabelEncoder()
smoker_encoder.fit(df['smoker'])
df['smoker-encoded'] = smoker_encoder.transform(df['smoker'])

In [10]:
# Create a one hot encoder and fit the available types of regions in the dataset
region_encoder = preprocessing.OneHotEncoder()
region_encoder.fit(df['region'].to_numpy().reshape(-1, 1))

# Transform all entries into the one hot encoded representation
encoded_regions = region_encoder.transform(df['region'].to_numpy().reshape(-1, 1)).toarray()

# Add each new encoded variable or feature to the dataset
for i, category in enumerate(region_encoder.categories_[0]):
    df[f'{category}-encoded'] = encoded_regions.transpose()[i]

## 2.2. Filtrado de variables

In [15]:
# Filtering or removing of non desired variables
df_x = df[['age', 'bmi', 'smoker-encoded', 'children', 'sex-encoded', 'northwest-encoded', 'northeast-encoded', 'southwest-encoded', 'southeast-encoded']]
df_y = df['charges']

# 3. Separación del conjunto de entrenamiento y evaluación

In [16]:
from sklearn import model_selection

In [17]:
from sklearn import preprocessing

## 3.1. Separación de los conjuntos
Es importante notar que, se realiza la separación del conjunto de datos original en **train**, **valid** y **test**, por fuera del framework de Keras para garantizar un adecuado tratamiento de los conjuntos acorde a la metodología empleada. En otras palabras, de esta forma nos aseguramos que cualquier preprocesamiento o normalización sobre validación (valid) y evaluación (test) se realiza a partir de la información obtenida en entrenamiento.

In [102]:
# Split the dataset into train_valid and test
x_train_valid, x_test, y_train_valid, y_test = model_selection.train_test_split(df_x, df_y, test_size=0.2, random_state=15, shuffle=True)

In [103]:
# Split the dataset into train and valid
x_train, x_valid, y_train, y_valid = model_selection.train_test_split(x_train_valid, y_train_valid, test_size=0.2, random_state=23, shuffle=True)

## 3.2. Normalización de variables

In [104]:
# Select the variables where the z-score will be applied
scalable_variables = ['bmi', 'age']

if scalable_variables:
    # Create an instance of the StandardScaler for each variable
    scaler = preprocessing.StandardScaler()

    # Fit the distribution
    scaler.fit(x_train.loc[:, scalable_variables])

    # Transform and normalize all variables
    x_train.loc[:, scalable_variables] = scaler.transform(x_train.loc[:, scalable_variables])
    x_valid.loc[:, scalable_variables] = scaler.transform(x_valid.loc[:, scalable_variables])
    x_test.loc[:, scalable_variables] = scaler.transform(x_test.loc[:, scalable_variables])

# 4. Regresión Lineal


#### Comentarios
1. Al principio, sucedió que el MAE era muy lento para convergencia, lo cual tiene sentido por el tipo de función de costo que representa. Particularmente, comparado con MSE, es mucho más lentro. Empecé probando modificar de forma estática y a mano el **learning rate**.
2. Luego, con un learning rate cada vez mayor, pude observar que el entrenamiento era más rápido, pero sucedían dos cuestiones. En primer lugar, que se producía una especie oscilación en torno a un valor que asumo que es el mínimo al cual se acerca el entrenamiento, con lo cual sería necesario disminuir cerca de ahí el valor del learning rate. Por otro lado, este mínimo no era el mismo mínimo que obtuve con el MSE, debe ser un plateau, un mínimo local pero no el absoluto. Me propuse usar **learning rate dinámico** y **comenzar de diferentes puntos**.
3. Cuando probe utilizar MSE, si no normalizaba con z-score todas las variables, rápidamente divergía la función de costo y se rompía el entrenamiento. Por otro lado, la misma normalización afectaba mucho al entrenamiento del MAE. *¿Por qué?* Lo pude corregir un poco al aumentar el learning rate por un factor, lo cual debe tener sentido si se considera que ahora las variables estando normalizadas tienen una menor magnitud lo cual puede producir que los pasos sean menores que antes, y por eso se ralentizó.
4. Interesante, llegué a esta discusión https://datascience.stackexchange.com/questions/9020/do-i-have-to-standardize-my-new-polynomial-features a raiz de una pregunta bastante sencilla, **¿por qué no está mejorando la métrica con mayor orden de polynomial features?**. Resulta ser que normalizando las variables y luego aplicando polynomial features, obtengo nuevas variables que siguen encontrándose en el intervalo [0,1] pero que su orden de magnitud es mucho menor. *Conclusión, siempre normalizar las variables que entran al modelo, y por ende si aplicas polynomial features tenés que normalizar luego de crear las nuevas variables.*

In [105]:
from src import rl_helper
importlib.reload(rl_helper);

In [106]:
mae = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                          learning_rate=1000,
                          scheduler='time-decay',
                          decay_rate=0.01,
                          epochs=500,
                          batch_size=32
                         )

Model logs at tb-logs/rl/20210524-174409
Model checkpoints at checkpoints/rl/20210524-174409
Model: "sequential_40"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_40 (Dense)             (None, 1)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________
Mean absolute error of the test set 3117.51220703125


In [69]:
mae = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                          learning_rate=10000,
                          scheduler='step-decay',
                          drop_rate=0.5,
                          epochs_drop=10,
                          epochs=500,
                          batch_size=128,
                         )

Model logs at tb-logs/rl/20210524-162945
Model checkpoints at checkpoints/rl/20210524-162945
Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 1)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________
Mean absolute error of the test set 3087.209716796875


In [71]:
mae = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                          learning_rate=10000,
                          scheduler='exponential-decay',
                          decay_rate=0.07,
                          epochs=500,
                          batch_size=128,
                         )

Model logs at tb-logs/rl/20210524-163101
Model checkpoints at checkpoints/rl/20210524-163101
Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 1)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________
Mean absolute error of the test set 3083.491455078125


In [83]:
mae = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                          learning_rate=1000,
                          scheduler='exponential-decay',
                          decay_rate=0.01,
                          optimizer='adam',
                          beta_1=0.9,
                          beta_2=0.99,
                          epochs=500,
                          batch_size=128,
                         )

Model logs at tb-logs/rl/20210524-164951
Model checkpoints at checkpoints/rl/20210524-164951
Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 1)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________
Mean absolute error of the test set 3061.910888671875


In [73]:
mae = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                          learning_rate=5000,
                          degree=2,
                          epochs=500,
                          batch_size=128,
                          scheduler='exponential-decay',
                          decay_rate=0.09
                         )

Model logs at tb-logs/rl/20210524-163215
Model checkpoints at checkpoints/rl/20210524-163215
Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 1)                 55        
Total params: 55
Trainable params: 55
Non-trainable params: 0
_________________________________________________________________
Mean absolute error of the test set 1715.5869140625


In [84]:
mae = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                          learning_rate=5000,
                          degree=2,
                          scheduler='exponential-decay',
                          decay_rate=0.09,
                          optimizer='adam',
                          beta_1=0.9,
                          beta_2=0.99,
                          epochs=500,
                          batch_size=128
                         )

Model logs at tb-logs/rl/20210524-165108
Model checkpoints at checkpoints/rl/20210524-165108
Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 1)                 55        
Total params: 55
Trainable params: 55
Non-trainable params: 0
_________________________________________________________________
Mean absolute error of the test set 1701.0277099609375


In [74]:
mae = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                          learning_rate=5000,
                          degree=3,
                          scheduler='exponential-decay',
                          decay_rate=0.1,
                          epochs=500,
                          batch_size=128
                         )

Model logs at tb-logs/rl/20210524-163327
Model checkpoints at checkpoints/rl/20210524-163327
Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 1)                 220       
Total params: 220
Trainable params: 220
Non-trainable params: 0
_________________________________________________________________
Mean absolute error of the test set 2615.01220703125


In [85]:
mae = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                          learning_rate=5000,
                          degree=3,
                          scheduler='exponential-decay',
                          decay_rate=0.1,
                          optimizer='adam',
                          beta_1=0.9,
                          beta_2=0.99,
                          epochs=500,
                          batch_size=128
                         )

Model logs at tb-logs/rl/20210524-165229
Model checkpoints at checkpoints/rl/20210524-165229
Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 1)                 220       
Total params: 220
Trainable params: 220
Non-trainable params: 0
_________________________________________________________________
Mean absolute error of the test set 1831.2984619140625


In [75]:
mae = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                          learning_rate=1000,
                          degree=4,
                          scheduler='exponential-decay',
                          decay_rate=0.1,
                          epochs=500,
                          batch_size=128
                         )

Model logs at tb-logs/rl/20210524-163443
Model checkpoints at checkpoints/rl/20210524-163443
Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 1)                 715       
Total params: 715
Trainable params: 715
Non-trainable params: 0
_________________________________________________________________
Mean absolute error of the test set 4261.63916015625


In [86]:
mae = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                          learning_rate=1000,
                          degree=4,
                          scheduler='exponential-decay',
                          decay_rate=0.1,
                          optimizer='adam',
                          beta_1=0.9,
                          beta_2=0.99,
                          epochs=500,
                          batch_size=128
                         )

Model logs at tb-logs/rl/20210524-165402
Model checkpoints at checkpoints/rl/20210524-165402
Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 1)                 715       
Total params: 715
Trainable params: 715
Non-trainable params: 0
_________________________________________________________________
Mean absolute error of the test set 2332.2734375


In [89]:
mae = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                          learning_rate=1000,
                          degree=4,
                          scheduler='exponential-decay',
                          decay_rate=0.01,
                          optimizer='adam',
                          beta_1=0.9,
                          beta_2=0.99,
                          epochs=500,
                          batch_size=256
                         )

Model logs at tb-logs/rl/20210524-165914
Model checkpoints at checkpoints/rl/20210524-165914
Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 1)                 715       
Total params: 715
Trainable params: 715
Non-trainable params: 0
_________________________________________________________________
Mean absolute error of the test set 2535.2734375


In [97]:
mae = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                          learning_rate=1.0,
                          degree=8,
                          optimizer='adam',
                          beta_1=0.9,
                          beta_2=0.99,
                          epochs=1000,
                          batch_size=64
                         )

Model logs at tb-logs/rl/20210524-171435
Model checkpoints at checkpoints/rl/20210524-171435
Model: "sequential_38"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_38 (Dense)             (None, 1)                 24310     
Total params: 24,310
Trainable params: 24,310
Non-trainable params: 0
_________________________________________________________________
Mean absolute error of the test set 6937.7080078125
